In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [1]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [1]:
data = train.copy()
data_test = test.copy()

- Survival = 0: Öldü 1: Kurtuldu
- Fare = Yolcu Ücreti
- Embarked = Bindirme Limanları
- Parch = Ebeveyn - Cocukların sayısı
- sibsp = Eş-Kardeşlerin sayısı 

In [1]:
feature_info = {'Survival_0':'Öldü',
               'Survival_1':'Kurtuldu',
               'Fare':'Yolcu Ücreti',
               'Embarked':'Bindirme Limanları',
               'Parch':'Cocukların Sayısı',
               'sibsp':'Kardeşlerin Sayısı',
                'Pclass':'Bilet Sınıfı'
               }

keys = list(feature_info.keys())
values = list(feature_info.values())

for i,j in zip(keys,values):
    print(i, ":", j)

- Datamızın tiplerini ve o tipin kaç tane değişkeni olduğunu bulduk

In [1]:
types = ['object','float','integer','bool']

for i in types:
    print(i + " Değişkenler:","\n", "Değişken Sayısı:",
         len(data.select_dtypes(i).columns), "\n", 
          data.select_dtypes(i).columns.tolist(), "\n")

In [1]:
data.head()

In [1]:
print(f"Train Shape:{data.shape}\nTest Shape:{data_test.shape}")

- **Survived** değişkendeki veri sayılarının,veri seti sayısına bölerek oranını buluyoruz

In [1]:
data.Survived.value_counts() / data.shape[0]

# Kategorik Değişken

In [1]:
data.info()

- **Object** tipine sahip değişkenlerimizi datamızdan alıyoruz

- Bunu 3 şekilde yapabiliriz.

In [1]:
#1. Yol

list(data.dtypes[data.dtypes == 'object'].index)

#Veya aşağıdaki gibi de yazabiliriz 

#data.dtypes[data.dtypes == 'object'].index.tolist()

In [1]:
# 2. Yol

object_cols = (data.dtypes == 'object')

object_cols = list(object_cols[object_cols].index)

object_cols

In [1]:
# 3. Yol : List Comprehension

[cols for cols in data.columns if data[cols].dtypes == 'object']

- Sadece **object** data tipine sahip olan değişkenler , **Kategorik Değişken** olacak diye bir kural yok.
- **Object** data tipine sahip olmayan değişkenlerde **Kategorik Değişken** olabilir.
- Bu tür değişkenlere ulaşmak aşağıda ki metodları kullanabiliriz.

- **Benzersiz Veri**' leri almak için 2 metodumuz vardır.
  - **unique( )**
  - **nunique( )**

- Değişkenlerimizdeki **Benzersiz Verileri** alacağız.Aşağıda ki örnekte; 
  - Değişkenimizde ki **Benzersiz Veri** sayımız 10'ten küçük ise onu alacağız

In [1]:
# 1- unique() kullanarak:

[cols for cols in data.columns if len(data[cols].unique()) < 10]

- Bir değişkende ki verilerin **benzersiz verileri** , **10'dan** küçük ise bunları kategorik veri olarak alıyoruz.

In [1]:
# 2- nunique() kullanarak:

cat_cols = [cols for cols in data.columns if data[cols].nunique() < 10]

cat_cols

- **Survived** , **Pclass** vb değişkenlerimiz bizim **object** data tipine sahip değişkenlerimiz olmamasına rağmen aslında **Kategorik Değişkenlerdir**.

- Benzersiz veriler ile belli değerden küçük ise kategorik veri kabul edeceğimiz değişkenleri alan ve görselleştiren bir fonskiyon elde edeceğiz.


In [1]:
import seaborn as sns
import matplotlib.pyplot as plt


def cate_features(df,number,plot = False):
    
    cat_cols = [cols for cols in df.columns if df[cols].nunique() < int(number)]
    
    for cols in cat_cols:
        
        data_new = pd.DataFrame({cols : df[cols].value_counts(),
                            'Oran':np.divide(df[cols].value_counts() ,len(df))})
    
        print(data_new, end = "\n\n\n\n")
        
        if plot: #True ise görselleştir.
            
            sns.countplot(x=cols, data=df)
            
            plt.show()
    
      
cate_features(df = data, number = 10, plot=True)

- Şimdi ise kaç belli değer altında kalan **Benzersiz Verileri** alarak kaç **Kateorik Değişken** olduğunu tutan bir fonksiyon yazacağız.

In [1]:
def categories_summary(df, number_features):
    
    cat_cols = [cols for cols in df.columns if df[cols].nunique() < int(number_features)]
    
    variable_count = 0 # Kategorik Değişkenleri sayar
    more_class = []    # Belirlediğimiz rakamdan fazla benzersiz verileri olan değişkenleri tutacağız.
   
    for col in df:
        
        if col in cat_cols:
            
            print(f"##########################################")
            print(pd.DataFrame({col:df[col].value_counts(),
                                    'Oran':df[col].value_counts() / len(df)}),
                 end = "\n\n\n")
            variable_count +=1     
        else:
            
            more_class.append(df[col].name) #Değişken ismini alır(Columm Name)
    
    print(f"Belirlenen Kategorik Değişken Sayısı = {variable_count}")
    print(f"Belirlediğimiz Sayıdan Yüksek Benzersiz Veriye Sahip Değişkenler = {more_class}")
        
    
categories_summary(data,number_features = 10)

# Sayısal Değer 

- **describe()** metodu default olarak datamızın 25. ,50. ,75. yüzdeliklerini verir.
- Fakat biz hangi yüzdelik dilimlerini almak istiyorsak **list içerisinde** bildirebiliriz.

In [1]:
data.describe().T

In [1]:
data.describe([0.01, 0.10, 0.80, 0.90, 0.95, 0.99]).T

- Kaç adet **Numeric** data tipine sahip değişkenlerimiz var onlara bakalım

In [1]:
numeric_cols = [cols for cols in data.columns if data[cols].dtypes != 'object' ]

numeric_cols

- **Survived** bizim hedef değişkenimiz olduğu için **numeric_cols** değişkenlerimiz arasında **olmamalı**.
- Aynı zamanda **PassengerId** de **numeric_cols** değerleri arasında olmamalı.. 

In [1]:
numeric_cols = [cols for cols in data.columns if data[cols].dtypes != 'object'
               and cols not in 'Survived'
               and cols not in 'PassengerId']

numeric_cols

- Bu numeric değişkenlerimizin **Histogram Grafiğini** çizdirelim

In [1]:
def NumHist(df,plot = False):
    
    num_cols = [cols for cols in data.columns if df[cols].dtypes != 'object'
               and cols not in 'Survived'
               and cols not in 'PassengerId'] #Numeric değişkenleri aldık.
    
    for cols in num_cols: 
        
        if plot: 
            
            print(f"################## {cols} ############################")
        
            df[cols].hist()
            plt.show() #Bu komut sayesinde tüm bir önceki histogram komutunu ekrana basar.
        
NumHist(data, plot = True)

# Hedef Değişken Analizi 

- Bizim Hedef değişkenimiz **Survived** deişkenimizdir.
- Şimdi ise datamızdaki diğer değişkenlerimiz ile hedef değişkenimiz olan **Survived** ile olan ilişkilerine bakacağız.

- Bir kategorik değişken için hedef değişken analizine bakacağız.
- ilk olarak **ortalama**sına bakalım

In [1]:
data.groupby('Sex')['Survived'].mean()

- Şimdi ise **male** ve **female** verilerinin **Survived** değerlerine bakalım

In [1]:
data.groupby('Sex')['Survived'].value_counts()

- Tüm **Kategorik Dğeişkenler** ile **Hedef Değişkenimiz** olan **Survived** ile ilişkilerini bize veren fonksiyon yazalım

In [1]:
def Cate_Target(df,number,hedef = 'Survived'):
    
    cat_cols = [cols for cols in data.columns if df[cols].nunique() < int(number)
               and cols not in hedef]
    
    for cat in cat_cols:
        
        dataNew = pd.DataFrame({'Hedef Oran':df.groupby(cat)[hedef].mean()})
        
        print(dataNew, end = "\n\n")
        print("#################################")

Cate_Target(data,number=10)

- **Kategorik Değişkenlerimizin**, **Hedef Değişkenimiz** de ki değerlerin sayısına bakalım.

In [1]:
cat_cols = [cols for cols in data.columns if data[cols].nunique() < 10 
           and cols not in 'Survived']

for cols in cat_cols:
    
    data_new = pd.DataFrame({'Counts':data.groupby(cols)['Survived'].value_counts()})
    
    print("######################################")
    print(data_new)
    


# Sayısal Değişken 

- İlk olarak **Sayısal Değişken**'lerimizi alalım

In [1]:
numeric_cols = [cols for cols in data.columns if data[cols].dtypes != 'object'
               and cols not in 'Survived']

numeric_cols

- Bir **Sayısal Değişken**'e göre **Hedef Değişken** analizini yaparken **agg()** fonksiyonunu kullanırız.

- **Survived** değişkenimizde ;

   - **Fare** değişkenimizin **Ortalama** değerlerini.
   - **Age** değişkenimizin **Minimum** değerlerini.
   - **SibSp** değişkenimizin **Maximum** değerlerini aldık **agg()** fonskiyonu ile.

In [1]:
data.groupby("Survived").agg({"Fare": "mean" , "Age":"min", "SibSp":"max"})

- **Sayısal Değişkenlerimizin**, birbiri arasında ki **Korelasyon**'ununa bakalım

In [1]:
data.corr()

- **Heatmap** ile görselleştirelim 

In [1]:
sns.heatmap(data.corr(),annot= True );

# Missing Values

- Datamızda ki **eksik değer** kontrolünü yaptık.
- Datamız da 1 tane bile eksik değer olduğu zaman **True** cıktısını alacağız

In [1]:
data.isnull().values.any()

- Eksik Değere Sahip sütunlar **True** diğerleri **False** olur.

In [1]:
data.isnull().any()

- Değişkenlerdemizde ki toplam **eksik değer** sayıları

In [1]:
data.isnull().sum()

- **Eksik Değer** olan Değişkenleri bize verir.

In [1]:
data.columns[data.isnull().any()]

- Datamızda ki **Eksik Değerler**'in toplam sayısı

In [1]:
data.isnull().sum().sum()

- Datamızda ki eksik değere sahip olan **Column**ları alıyoruz.

In [1]:
#1. Yol : Normal For döngüsü ile alma.

miss = []

for cols in data.columns:
    
    if data[cols].isnull().any(): #True döneceğinden aşağıa inip oradaki kodu çalıştırır.
        
        miss.append(cols)
        
miss

In [1]:
#2. Yol : List Comprehension kullanarak alma.

missing_cols = [cols for cols in data.columns if data[cols].isnull().any()]

missing_cols

In [1]:
# 3. Yol : 

data.columns[data.isnull().any()].tolist()

- **Eksik değere** sahip sütunlarımızın ; 
 - Kaç tane eksik değeri olduğunu
 - Eksik değere sahip sütunların tiplerini hepsinden oluşan bir **DataFrame** elde ettik.
 
 
- Num_missing değeri 0'dan büyük olan değerleri aldık.
- Yani sadece eksik değere sahip olan değişkeni aldık.
- index ismini de Feature ile değiştirdik. 


- Seaborn ile featurelarımızın eksik değer oranını grafikleştirdik.
- **plot = False** default olarak grafiği çizmeyecek.Fonksiyon da **True** dediğimiz zaman  grafiğimizi çizecek
 

In [1]:
import seaborn as sns

def miss_value(df, plot = False ): 
    
    miss = pd.DataFrame({'Num_missing':df.isnull().sum(),
                       'Missing Ratio': np.divide(df.isnull().sum(), df.shape[0])}).sort_values("Num_missing", ascending = False )
                        
    miss['Datatypes'] = df[miss.index].dtypes.values
    
    miss = miss[miss['Num_missing'] >  0 ].reset_index().rename({'index':'Feature'}, axis = 1)
    
    print(f"Number of Missing Features : {len(miss['Feature'])}")
    
    if plot: #Bu şekilde True anlamına geliyor.
        
        sns.barplot(x = miss['Feature'], y = miss['Missing Ratio'])
        
    return miss
                

miss_value(data,plot = True)

- En az bir tane **Eksik Değere** sahip olan **Gözlem Birimleri**

In [1]:
data[data.isnull().any(axis=1)]

- Eksik değer olmayan değişkenler **True**, Eksik Değere Sahip olan değişkenler ise **False** değerlerini alır.

In [1]:
data.notnull().all()

- Tam olan **Gözlem Birimleri**

In [1]:
data[data.notnull().all(axis=1)]

- Belli yaş üzerindeki insanların sayısı ve  belli yaş altında ki insanların sayısını hesapladığımız bir **AgeCount** sınıfı oluşturduk.

In [1]:
class AgeCount():
    
    def __init__(self,value,df):
        
        self.value = value 
        self.df = df
        
    def AgeBack(self):
        
        back_count = int(self.df['Age'].describe()[0]) - self.df['Age'].gt(self.value).sum()
        
        back_cache = {'Person Number':back_count}
        
        return back_cache
    
    def AgeForward(self):
        
        for_count = self.df['Age'].gt(self.value).sum()
        
        for_cache = {'Person Number':for_count}
        
        return for_cache

- 30 yaş üzerindeki insanların sayısı

In [1]:
ky_age = AgeCount(30,data)

ky_age.AgeForward()

- 30 yaş altındaki insanların sayısı

In [1]:
ky_age.AgeBack()

- Name değişkeninden **Mr**, **Mrs** vb. alarak yeni bir **değişken** elde ediyoruz.
- **'([A-Za-z]+)\.'** bunun sayesinde isimlerin önündeki sıfatları alıyoruz.

In [1]:
data['New_Name'] = data['Name'].str.extract('([A-Za-z]+)\.', expand = False)

data.head(n=8)

- **" Age "** değişkeninde ki **eksik değerleri** , **" New_Name "** değişkeninde ki değerlerin **Age Ortalamasına** göre doldurduk

In [1]:
#New_Name değişkeninde ki değerlerin Age ortalamaları

data.groupby('New_Name')['Age'].mean()

- Elde ettiğimiz **New_Name** değişkenindeki değerlerin **Age** ortalamasına göre değiştirdik.
- Example:
  - Mr --> Age.mean == 32 ise **New_Name** değişkenin de **Mr** olan değer yerine 32 ile değiştirmiş oluyoruz.
  - **transform()** (dönüşüm)

In [1]:
data.groupby('New_Name')['Age'].transform('mean')

- Age değişkeninde ki **eksik değerleri(missing_values)**, New_Name değişkeninde ki Age ortalamalarına göre doldurduk.

In [1]:
data['Age'] = data['Age'].fillna(data.groupby('New_Name')['Age'].transform('mean'))

- Aşağıda gördüğümüz gibi **Age** değişkeninde ki NaN değerler yok.

In [1]:
data['Age'].isnull().any()

- **Embarked** sütununda ki eksik değerleri **en sık** olan  **S** değeri ile doldurduk.
- **Mode** alarakta en sık olan değeri doldurabilirz

In [1]:
#1. Yol Mode alarakta yapabiliriz

data['Embarked'].mode()[0]

- 2. Yol ise Sklearn içersinden **SimpleImputer** kullanarak yapabiliriz.
- **strategy**  = **most_frequently** diyerek sık kullanılan değer ile doldur diyoruz.
- Aslında mode almış oluyoruz
- Eksik değere sahip olan değişkenlerimizin eksik değerlerini, **En Sık olan veri** ile dolduran fonskiyon yazalım

In [1]:
from sklearn.impute import SimpleImputer

def category_counts(cols):
    
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
    
    data[cols] = imputer.fit_transform(data[[cols]])
    
    return data[cols].describe()[0]

category_counts('Embarked')

In [1]:
data['Embarked'].isnull().any()

# Feature Engineering

- Datamızda ki **New_Name** değişkeninde ki **Dr** değerlerinin yaş değerlerini aldık

In [1]:
data['Age'][data['New_Name'] == 'Dr']

- İstersek **Dr** değerlerinin **Age** ortalamalarını da alalım

In [1]:
data['Age'][data['New_Name'] == 'Dr'].mean()

- **Dr** ünvanına sahp kişilerin **Survived** sayılarına bakalım.

In [1]:
data['Survived'][data['New_Name'] == 'Dr'].value_counts()

- **Age** değişkenimizde ki değerlere göre **Young, Mature ve Old** değerleri oluşturduk.
- Bu değerlerle **New_Age_Count** adlı yeni değişken oluşturduk.

In [1]:
data.loc[(data['Age'] < 18), 'New_Age_Count'] = "Young" #New_Age_Count değişkeni için , Age değeri 18 den küçük değerler için "YOUNG" değeri elde ettik

data.loc[(data['Age'] >= 18) & (data['Age'] < 55 ), 'New_Age_Count'] = "Mature" #New_Age_Count değişkeni için , Age değeri 18den büyük 55 den küçük değerler için "MATURE" değeri elde ettik

data.loc[(data['Age'] >= 55 ), 'New_Age_Count'] = 'Old' #New_Age_Count değişkeni için , Age değeri 55 den büyük değerler için "OLD" değeri elde ettik

data.head(n=8)

- **Young** verisine sahip olan değerlerin hangi yaştan kaç kişi var ona baktık

In [1]:
data['Age'][data['New_Age_Count'] == 'Young'].value_counts()

- 18 yaşından küçüklerin **Survived** değşkeninde ki sayılarına baktık.

In [1]:
data['Survived'][data['New_Age_Count'] == 'Young'].value_counts()

- **SibSp** ve **Parch** değişkenlerini kullanarak **Alone** değişkeni elde edeceğiz.
- **SibSp(Kardeşlerin sayısı)** ile **Parch(Çocukların sayısı)** **0**'dan büyük ise **Alone** değişkeninde **0** olsun dedik.
- **SibSp(Kardeşlerin sayısı)** ile **Parch(Çocukların sayısı)** **0**'a eşit ise **Alone** değişkeninde **1** olsun dedik.

In [1]:
data.loc[((data['SibSp'] + data['Parch']) > 0  ), 'Alone'] = 0 
data.loc[((data['SibSp'] + data['Parch']) == 0 ), 'Alone'] = 1

data.head(n=8)

- **Fare** değişkenindeki değere göre **Fare_0** değişkeni elde ediyoruz.
- **Fare** değişkenindeki değer 1'den küçük ise **Fare_0** değişkenin de 1 olsun dedik, değil ise 0 olsun dedik.
- Yani böylelikle ücret ödeyen ve ödemeyen olarak sınıflandırmış olduk.

In [1]:
data['Fare_0'] = np.where(data['Fare'] < 1, 1, 0)

data.head(n=8)

- **Fare** değişkenindeki 1 'den küçük değerlere yani **Ücret ödemeyen** indexlerine ve sayısına bakalım

In [1]:
print(data['Fare'][data['Fare_0'] == 1], end = "\n\n")
print(f"Fare değişkenin de 0 olan gözlemlerin sayısı : {len(data['Fare'][data['Fare_0'] == 1])}")

# Data Visualization

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
sns.distplot(data.Fare,bins=20);

In [1]:
sns.catplot(x='Pclass',y='Fare',data=data);
sns.catplot(x='Survived',y='Fare',data=data);

- In general,have high social-economics values people is survived
- People is survived that pay high passengers price

In [1]:
sns.catplot(x='Pclass',y='Fare',hue='Survived',data=data);

In [1]:
sns.countplot(data['Pclass']);
print(f"Most Used Value: {data.Pclass.mode()[0]}")

In [1]:
sns.countplot(data['Survived']);
print(f"Most Used Value: {data.Survived.mode()[0]}")

In [1]:
fig, ax = plt.subplots(1,2,figsize=(12,8))

ax[0].set_title('of siblings-spouses aboard')
sns.countplot(data['SibSp'],ax=ax[0]);
ax[1].set_title('Children Number')
sns.countplot(data['Parch'],ax=ax[1]);

- Sosya-Ekonomik durumu iyi olan insanlardan sadece biri 4 çocuk sahibi.O da 250 civarı ücret ödemiş bu değer de Ödenen en yüksek bilet fiyatının yarısı

In [1]:
sns.catplot(x='Parch',y='Fare',hue='Pclass',data=data);

- EMBARKED

In [1]:
sns.countplot(data['Embarked']);

- The number of people getting on from city Q is fewer than other stops. Those who get from city Q are people who pay less.
- And the wages he pays are lower than the average wages.
- Those who pay the high passenger fares are from city C.

In [1]:
def Percentage(top,ort):
    
    result = 0
    y = 0
    y = np.divide(100,top)
    result = np.multiply(y,ort)
    
    return f"Percentage : {int(result)}"

- 0  ve 1 için ölüm oranı hesaplamak için ortalama alan fonskiyon yazalım

In [1]:
def Percentage_2(x,y):
    
    toplam = 0
    z = 0
    result = 0
    
    toplam = np.add(x,y)
    
    z = np.divide(100,toplam)
    
    result = np.multiply(z,y)
    
    return f"Percentage : {int(result)}"
     

In [1]:
embark_fare = data.groupby('Embarked')['Fare'].mean()
print(f"Passenger Price Mean:{data.Fare.mean()}\n{embark_fare}")

- En az yolcu ücreti alan liman **Q** , en fazla ölüm oranına sahip liman ise **S**

In [1]:
#limanlara göre ölüm oranı 

print(f"C : {Percentage(168,75)}\nQ : {Percentage(77,47)}\nS : {Percentage(646,427)}")

- PCLASS
- Bilet Sınıfı kötü olan yolcuların ölüm oranları daha fazla 

In [1]:
#Bilet sınıfına göre ölüm oranları 

print(f"Class 1 : {Percentage_2(136,80)}\nClass 2 : {Percentage_2(87,97)}\nClass 3 : {Percentage_2(119,372)}")

In [1]:
sns.catplot(x='Embarked',y='Fare',hue='Survived',data=data);

In [1]:
Pclass_Fare = data.groupby('Pclass')['Fare'].mean()

Pclass_Fare

In [1]:
Pclass_sur = data.groupby('Pclass')['Survived'].value_counts()

pd.DataFrame(Pclass_sur)

In [1]:
sns.countplot(data['Sex']);

- Cinsiyete göre Ölüm Oranları 

In [1]:
male  = data['Sex'].value_counts()[0]
female = data['Sex'].value_counts()[1]

print(f"Male:{male}\nFemale:{female}")

# Label Encodig 

- **Sex** değişkenimizde ki bulunan **Benzersiz Verilerimizin** kaç adet olduğuna bakalım

In [1]:
data['Sex'].value_counts().count()

- Kategorik Değişkenlerimizden max **2 tane veri** olan **Kategorik Değişkenlerimize** Label Encoding uyguladığımız fonksiyon yazalım. 

In [1]:
[cols for cols in data.columns if data[cols].dtypes == 'object'
               and len(data[cols].value_counts()) == 2]

In [1]:
from sklearn.preprocessing import LabelEncoder

lab_enc = LabelEncoder()

def Label(df):
    
    cat_cols = [cols for cols in data.columns if df[cols].dtypes == 'object'
               and len(df[cols].value_counts()) == 2]
    
    for cols in cat_cols:
        
        df[cols] = lab_enc.fit_transform(df[cols])
    
    return df.head(n=8) 

Label(data)   

- **Cabin** değişkenimize de LabelEncoder işlemi uygulayacağız.
- **Cabin** değişkenimizde 2'den farklı Benzersiz veri var.Ama yine de LabelEncoder uygulayacağız.
- **Cabin** değişkenini silebilirdik.Fakat **Cabin** değişkeni bizim **Hedef Değişkenimiz** üzerinde etksi vardır.
- O yüzden **unknown** değerleri tek değer diğer kabin değerlerini ise sırayla bir sayıya atamış olacağız

In [1]:
from sklearn.preprocessing import LabelEncoder

lab_enc = LabelEncoder()

data['Cabin'] = lab_enc.fit_transform(data['Cabin'])

data.head()

# One Hot Encoder

- One Hot Encoding işlemini yaparken **pd.get_dummies()** yöntemini kullanacağız.

In [1]:
pd.get_dummies(data, columns = ["Embarked"]).head()

In [1]:
pd.get_dummies(data, columns = ['Embarked'], prefix = "E" , drop_first= True).head()

In [1]:
data.head(n=8)

- 2'den fazla eşsiz veriye sahip olan **Kategorik Değişkenlrimize** için **One Hot Encoding** uygulayan fonksiyon uygulayalım 

In [1]:
pd.get_dummies(data,columns=["Embarked"],drop_first = True,dummy_na = False).head(n=8)

In [1]:
[cols for cols in data.columns if data[cols].dtypes == 'object'
               and len(data[cols].value_counts()) < 10]

In [1]:
[cols for cols in data.columns if data[cols].dtype != 'object']

In [1]:
[cols for cols in data.columns if data[cols].dtypes != 'object'
and data[cols].value_counts().count() < 10]

In [1]:
def OneHot(df, number, drop_first = False):
    
    cat_cols = [cols for cols in data.columns if data[cols].dtypes == 'object'
               and len(data[cols].value_counts()) < int(number)] # data[cols].value_counts().count() < 10 da yazabilirdik.
    
    df = pd.get_dummies(df, columns = cat_cols, drop_first = drop_first)
    
    return df

OneHot(data, number=10, drop_first = True) #datamıza eşitlemezsek işlemler datamız üzerinde kayıtlı olmuyor.   

In [1]:
data = OneHot(data,number=10, drop_first=True)

In [1]:
data.head(n=8)

In [1]:
cols = ['Pclass','New_Name']

data = pd.get_dummies(data, columns = cols , drop_first = True)

data.head()

# Outliers 

- **Aykırı Değerler** için **Numeric Feature(Sayısal Değişkenlerimiz)**'larımıza bakacağız.

In [1]:
num_cols = [cols for cols in data.columns if data[cols].dtypes != 'object'
           and cols not in 'Survived']

num_cols

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt 

sns.boxplot(data['PassengerId']);

- **Sayısal Değişkenimizin**, **Boxplot**'unu çizdiren fonksiyon yazalım.

In [1]:
def box_num(df, plot=False):
    
    num_cols = [cols for cols in data.columns if data[cols].dtype != 'object'
               and cols not in 'Survived'
               and cols not in 'PassengerId']
    
    for cols in num_cols:
        
        if plot:
            sns.boxplot(df[cols]);
            plt.show()
                

box_num(data, plot = True)

- **Fare** ve **Age** değişkenlerimizin **Aykırı Değerlerinin** fazla olduğunu görüyoruz.

- Aykırı değerler için eşik değerleri belirleyen fonskiyon yazalım.

In [1]:
def Outliers_Thres(df,cols):
    
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    
    IQR = np.multiply(1.5, np.subtract(Q3, Q1))
    
    alt_sınır = Q1 - IQR
    üst_sınır = Q3 + IQR
    
    #cache = {'alt_sınır':alt_sınır,'üst_sınır':üst_sınır}
    
    return alt_sınır, üst_sınır #fonksiyonumuz alt_sınır ve üst_sınır return ediyor
  
Outliers_Thres(data, cols = 'Age')    

- **alt ve üst eşik değerlerini** kullanmak istiyorsak fonskiyonumuzu aşağıdaki gibi çağırabiliriz.
- Fonksiyonumuz **2 adet değer return ettiği** için onları da **alt,üst** şeklinde fonskiyondan almış olduk.

In [1]:
alt, üst = Outliers_Thres(data, cols = 'Age') 

print(f"Alt Eşik Değeri = {alt}\nÜst Eşik Değeri = {üst}")

- Değişkenlerimiz için **Aykırı Gözlem olup olmadığının** analizini yapan fonksiyon yazalım

In [1]:
def Outliers_Feature(df, cols):
    
    value = []
       
    alt_eşik, üst_eşik = Outliers_Thres(df, cols)
    
    if df[(df[cols] < alt_eşik) | (df[cols] > üst_eşik)].any(axis=None):
        
        value.append("Yes")
    else:
        value.append("No")
  
    return value

Outliers_Feature(data, 'Age') 

- **Survived** ve **PassengerId** değişken dışında ki tüm **Sayısal Değişkenleri** alıyoruz.

In [1]:
num_cols = [cols for cols in data.columns if data[cols].dtype != 'object'
and cols not in 'Survived'
and cols not in 'PassengerId']

num_cols

- **Outliers_Feature** fonskiyonu kullanarak datamızda ki tüm **Sayısal Değişkenler**' e eksik değer kontrolünü kendisi bakan fonksiyon yazalım.

In [1]:
def Auto_Outliers(df, columns):
    
    True_feature = [] #Aykırı Gözlemi olan Değişkenleri tutacak
    
    False_feature = [] #Aykırı Gözlem olmayan Değişkenleri tutacak
    
    for cols in columns:
        
        value = Outliers_Feature(df,cols) #Bir önceki fonskiyonumuzu kullanıyoruz
        
        for i in value:
            
            if i.startswith("Y"):
                
                True_feature.append(cols)
            else:
                False_feature.append(cols)
                  
    print(f"Aykırı Gözlem olan Değişkenler = {True_feature}\nToplam Aykırı Gözlem Değişken Sayısı = {len(True_feature)}", end = "\n")
    print(f"Aykırı Gözlem olmayan Değişkenler = {False_feature}\nToplam Aykırı Gözlem Olmayan Değişken Sayısı = {len(False_feature)}", end = "\n") 
             
Auto_Outliers(data, num_cols)    

- **Alt Eşik** ve **Üst Eşik** değerlerini elde ettiğimiz fonksiyonumuz ile **Age** değişkenin alt ve üst eşik değerlerini buluyoruz.

In [1]:
Outliers_Thres(data, 'Age')

- **Age** değişkenimizde ki **Aykırı Değerlerimiz** 'in olduğu gözlemleri **Tüm Datamız** üzerinden bakıyoruz.

In [1]:
data[(data['Age'] < alt) | (data['Age'] > üst)]

- Yukarıdaki Gözlem birimleri dışındaki tüm gözlem birimlerini aşağıda ki gibi görebiliriz.
- Yani **Aykırı Gözlem** olmayan **Tüm Gözlemler**

In [1]:
data[~((data['Age'] < alt) | (data['Age'] > üst))]

- Sadece **Age** değişkenimizdeki **Aykırı Gözlemlere** bakıyoruz.

In [1]:
data['Age'][(data['Age'] < alt) | (data['Age'] > üst)]

- Sadece **Age** değişkenin de **Aykırı Değerler Dışında ki** diğer verileri alalım

In [1]:
data['Age'][~((data['Age'] < alt) | (data['Age'] > üst))]

- **Auto_Outliers** fonksiyonumuza , **Aykırı Değere Sahip** olan Değişkenlerin **Toplam Aykırı Değer** sayısını da yazdırma özelliği ekleyelim.

In [1]:
def Auto_Outliers(df, columns):
    
    True_feature = [] #Aykırı Gözlemi olan Değişkenleri tutacak
    
    False_feature = [] #Aykırı Gözlem olmayan Değişkenleri tutacak
    
    number_cols = [] #Değişkenlerin Toplam Aykırı Gözlem Sayılarını tutacak
       
    for cols in columns:
        
        alt ,üst = Outliers_Thres(df, cols) #alt_eşik ve üst_eşik değerlerini veren fonskiyon
        
        value = Outliers_Feature(df,cols) #Aykırı Gözlem kontrolü yapan fonskiyon
        
        for i in value:
            
            if i.startswith("Y"):
                
                True_feature.append(cols)
                
                number = df[cols][(df[cols] < alt) | (df[cols] > üst)].value_counts().count()
                
                number_cols.append((cols,number)) #List içerisine tupple şeklinde 2 değer ekleyebiliyoruz.
                
            else:
                False_feature.append(cols)  
                  
    print(f"Aykırı Gözlem olan Değişkenler = {True_feature}\nToplam Aykırı Gözlem Değişken Sayısı = {len(True_feature)}", end = "\n")
    print(f"Aykırı Gözlem olmayan Değişkenler = {False_feature}\nToplam Aykırı Gözlem Olmayan Değişken Sayısı = {len(False_feature)}", end = "\n") 
    
    for i,j in number_cols: #number_cols listemizde tupple olarak 2 değerler olduğu için i,j ile o 2 değerleri alıyoruz.
        
        print("###################################",end = "\n")
        print(f"\nDeğişken : {i}\nToplam Aykırı Değer Sayısı : {j}",end = "\n\n")
        
        
    
Auto_Outliers(data, num_cols)

- En çok **Aykırı Değere** sahip olan Değişkenlerimiz; 

  - **Fare** , **Age** 
 - Bu değişkenlerimizin **Aykırı Değerlerini** alt ve üst eşik değerlerine baskılayacağız
 - Bu yüzden bu değişkenlerimizin alt ve üst eşik değerlerini hesaplayacağız.
 - Bunun için alt ve üst eşik belirleyen fonskiyonumuz **Outliers_Thres**'i kullanacağız.
 - **Parch** ve **SibSp** değişkenlerimiz kategorik değişken olarak algıladığımız için onların Aykırı Değerleri yoktur.

- Örneğin ; **SibSp** değişkenimizin boxplotunu çizdirdiğimizde sol taraftai değerler aslında değişkenin benzersiz değerleridir.
- Değişkenin ortalaması 0'a çektiği için Değişkenin 8,6,5 deki verileri aykırı olarak görüyor

- Aynı durum **Parch** Değişkenimiz için de söz konusudur.

In [1]:
sns.boxplot(data=data['SibSp']);

In [1]:
data['SibSp'].value_counts()

- Aykırı Değerlerini alt ve üst eşik değerlerine baskılayacağımız Değişkenlerimiz için **Alt Eşik ve Üst Eşik değerleri** buluyoruz.

In [1]:
out_feature = ['Age','Fare']

for feature in out_feature:
    
    print(f"{feature} : {Outliers_Thres(data, cols = feature)}")  #Outliers_Thres fonskiyonumuz ile alt ve üst eşik değerleri elde ediyoruz.

In [1]:
alt, üst = Outliers_Thres(data,'Age') 
print(alt,üst)

- **Age** değişkenimizin alt eşik değeri altında kalan aykırı değere ulaşalım

In [1]:
data['Age'][data['Age'] < alt]

- **Age** değişkenimizin **Üst Eşik Değeri** üstünde kalan aykırı değerlere ulaşalım

In [1]:
üst_değer = data['Age'][data['Age'] > üst]

üst_değer

- **Age ve Fare** değişkenlerimiz için **toplam alt ve üst aykırı değerleri** bulan fonskyon yazalım

In [1]:
def low_up_outliers(df,feature, plot = False):
    
    for cols in feature:
        
        alt_eşik , üst_eşik = Outliers_Thres(df, cols=cols) #alt ve üst eşik değeri belirleyen fonskiyon
        
        alt = df[cols][df[cols] < alt_eşik]
        
        üst = df[cols][df[cols] > üst_eşik]
        
        if plot:
            
            print(f"################### {cols} #################")
            sns.boxplot(df[cols])
            plt.show()
        
        print(pd.DataFrame({'Değişken':cols,
                           'Toplam Alt Aykırı Değer':alt.shape,
                           'Toplam Üst Aykırı Değer':üst.shape
                           }), end = "\n\n")

        
feature = ['Age','Fare']
low_up_outliers(data, feature, plot = True)

- **Aykırı Değerlerimiz** için ;

  - **Alt Aykırı Değerler** için alt aykırı değere baskılayacağız.
  - **Üst Aykırı Değerler** için üst aykırı değere baskılayacağız.
  
- Değişkenlerimiz için Aykırı Değer Hesaplayan,
- Aykırı değerleri alt ve üst değerlere göre baskılayan bir fonksiyon yazacağız.
- **Fare** değişkenimizin **alt eşik değeri altında kalan aykırı değeri olmadığı** için **Fare değişkenimizi alt aykırı değerine baskılamayacağız**.
- O yüzden fonskiyonumuzda bir **if koşulu** koyduk.
- Fonskiyonmuz en son bize değişkenlerdeki **toplam aykırı değerleri** verecek.
- Ve En sonunda **Aykırı Değişkenlere** sahip değişkenlerimizin alt ve üst eşik değerlerine baskıladıktan sonra ki boxplotunu çizdirecek.

In [1]:
def press_outliers(df, features,plot = False):
    
    for cols in features:
        
        alt_eşik,üst_eşik = Outliers_Thres(df, cols=cols) #alt ve üst eşik değerleri belirlediğimiz fonskiyon
        
        df[cols][df[cols] > üst_eşik] = üst_eşik
        
        if cols.startswith('A'): #Age değişkenini alt aykırı değerine baskıladık.Fare Değişkenini baskılamadık !!!
            
            df[cols][df[cols] < alt_eşik] = alt_eşik
        
        if plot:
            
            print(f"################# {cols} ###################")
            sns.boxplot(df[cols])
            plt.show()
        
        print(pd.DataFrame({'Değişken' : cols,
                           'Toplam Alt': df[cols][df[cols] < alt_eşik].shape,
                           'Toplam Üst': df[cols][df[cols] > üst_eşik].shape
                           }), end=" \n\n")
        
        
features = ['Age','Fare']   

press_outliers(data,features, plot = True)                

# Data Preprocessing 

In [1]:
data.head()

- **PassengerId, Name, Ticket** değişkenlerini Datamızdan çıkarıyoruz

In [1]:
drop_features = ['PassengerId','Name','Ticket']

data = data.drop(drop_features, axis=1)

data.head()

- **Bağımlı Değişkenimiz** olan **Survived** değişkenimizi datamızdan alıyoruz.

In [1]:
survived = data['Survived']

survived

- Survived değişkenimizi datamızdan cıkarıyoruz.Böylelikle **Bağımsız Değişkenimizi** de elde etmiş oluyoruz.

In [1]:
data = data.drop('Survived', axis=1)

data.head()

In [1]:
test.head()

In [1]:
test['New_Name'] = test['Name'].str.extract('([A-Zaz]+)\.', expand = False)

test.head()

- **Bağımlı ve Bağımsız Değişkenlerimizi** kullanarak **train ve test** verilerimizi elde edeceğiz.

In [1]:
from sklearn.model_selection import train_test_split

def Train_Test(data_x, data_y, size ):
    
    x_train,x_test,y_train,y_test = train_test_split(data_x, data_y, test_size = size, random_state = 18)
    
    print(f"X_train:{x_train.shape}\nX_Test:{x_test.shape}\nY_train:{y_train.shape}\nY_test:{y_test.shape}")
    
    cache = {'x_train':x_train,
             'x_test':x_test,
             'y_train':y_train,
             'y_test':y_test}
    
    return cache

cache = Train_Test(data,survived,size=0.33)

- Olusturduğumuz train ve test datalarımızı alalım

In [1]:
x_train,x_test,y_train,y_test = cache['x_train'],cache['x_test'],cache['y_train'],cache['y_test']

In [1]:
x_train

In [1]:
y_train

- **x_train ve x_test** datalarımızda **Age ve Fare** değişkenlerimize **MinMaxScaler** işlemi uygulayan fonskiyon yazalım.

In [1]:
from sklearn.preprocessing import MinMaxScaler

def MinMax(x_train, x_test,feature):
    
    scaler = MinMaxScaler()
    
    for cols in feature:
        
        x_train[cols] = scaler.fit_transform(x_train[[cols]])
        x_test[cols] = scaler.fit_transform(x_test[[cols]])
        
    cache_scaler = {'x_train':x_train,
                    'x_test':x_test
                   }

    return cache_scaler
    
features = ['Age','Fare']

cache_scaler = MinMax(x_train,x_test,features)

In [1]:
x_train, x_test = cache_scaler['x_train'],cache_scaler['x_test']

In [1]:
x_train

In [1]:
x_test

# Model


In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

- Sınıflandırma Algoritmalarımız kullanarak modelimiz için en iyi algoritmayı bulacağız.
- Tüm Algortimaları fit eden ve score elde eden ve bu scoreları bize vereb fonksiyon yazalım


In [1]:
models = [('LR',LogisticRegression()),
         ('SVC',SVC()),
         ('DTC',DecisionTreeClassifier()),
         ('KNN',KNeighborsClassifier()),
         ('RF',RandomForestClassifier()),
         ('GB',GradientBoostingClassifier()),
         ('XGB',XGBClassifier())]

def Model(data_x, data_y, cv):
    
    scores = []
    
    names = []
    
    for name,model in models:
        
        cross = cross_val_score(model, data_x, data_y, cv=cv, scoring = 'accuracy')
        
        scores.append((name,cross.mean()))
        
    return scores
    
    
Model(x_train,y_train,cv=10) 

- En İyi modelimizin **GradientBoostingClassifier** olduğunu gördük.

# Model Tuning 

- **GradientBoostingClassifier** modelimiz için en iyi parametrelerini bularak o parametreler ile yeni bir **GradientBoostingClassifier** modeli oluşturacağız

In [1]:
gbc_model = GradientBoostingClassifier()

gbc_model.fit(x_train,y_train)

- **GradientBoostingClassifier** modelimiz için en iyi parametrelerini bularak o parametreler ile yeni bir **GradientBoostingClassifier** modeli oluşturacağız

- Modelimiz için en iyi parametreleri denemesini istediğimiz parametre değerlerini modele vereceğiz o da modelimiz için en iyi değerleri bize verecek.

- Bu değerler ile yeni bir **GradientBoostingClassifier** modeli elde edeceğiz

In [1]:
from sklearn.model_selection import GridSearchCV

def GridSearch(data_x, data_y, model, params, cv):
    
    grid_search = GridSearchCV(estimator = model, param_grid = params, cv=cv) #GridSearchCV modelimizi oluşturuyoruz.
    
    grid_search.fit(data_x,data_y) #GridSearchCV modelimizi fit ediyoruz
    
    best_params = grid_search.best_params_ #Modelimiz için en iyi parametreleri alacağız
    
    return best_params

gbc_params = {'learning_rate':[0.001,0.01,0.1,0.5], #Modelimiz için en iyi parametreleri bulmasını istediğimiz parametre değerleri
              'max_depth':[2,6,8,10],
              'min_samples_split':[2,6,8,10]}

GridSearch(x_train, y_train, model = gbc_model, params = gbc_params, cv=10)

- Bulduğumuz **En iyi Parametreler** yeni bir **GradientBoostingClassifier** modeli oluşturacağız.

In [1]:
def Model_Tuned(data_x, data_y, data_test, l_rate, m_depth, ms_split):
    
    gbc_tuned = GradientBoostingClassifier(learning_rate=l_rate, max_depth= m_depth, min_samples_split = ms_split)
    
    gbc_tuned.fit(data_x, data_y)
    
    y_pred = gbc_tuned.predict(data_test)
    
    cache_tuned = {'y_pred':y_pred}
    
    return cache_tuned

cache_tuned = Model_Tuned(x_train, y_train, x_test, l_rate=0.1, m_depth =2, ms_split= 10)

In [1]:
y_pred = cache_tuned['y_pred']
y_pred

In [1]:
from sklearn.metrics import classification_report 

print(classification_report(y_pred, y_test))